# The Algorithm

In [1]:
import sympy

Step 1: get $h(x,y)$ from $f(x,y)$ and $g(x,y)$, remembering that $h(x,y) = x(g\partial_xf - f\partial_xg) - y(g\partial_yf - g\partial_yg)$

In [2]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
def hgetter(f,g):
    return x*(g*sympy.diff(f, x) - f*sympy.diff(g, x)) + y*(g*sympy.diff(f, y) - f*sympy.diff(g, y))

In [3]:
f = sympy.Poly('x**2 + 2*x*y + y**2')
g = sympy.Poly('x**3 + y')

In [5]:
h = hgetter(f,g)
print(h)

x*Poly(-x**4 - 4*x**3*y - 3*x**2*y**2 + 2*x*y + 2*y**2, x, y, domain='ZZ') + y*Poly(2*x**4 + 2*x**3*y - x**2 + y**2, x, y, domain='ZZ')
